In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
import sqlite3 as db
import geopy
#import matplotlib.pyplot as pp

In [92]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [93]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [66]:
def stopQualityMask(comName: str,closestStop: str):
    cnx = connectDB()
    try:
        gap = dt.date.today().year-2018
        query = f'SELECT COUNT(DISTINCT(year)) as "frq" FROM coefficients_bySpecies WHERE comName = "{comName}" AND closestStop = "{closestStop}"'
        coeficients = pd.read_sql(query,con=cnx)
        coeficients['frq'] = coeficients.apply(lambda g: (g.frq/gap),axis=1)
    except Exception as maskExc:
        raise maskExc
    return coeficients['frq']

In [257]:
def weeklyAbundance():
    cnx = connectDB()
    try:
        #note that the query filters out birds only seen by two people
        ObsDataset = pd.read_sql('SELECT comName,FX.locId,StopName,obsDt,howMany,subId FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.comName in ("Canada Jay","Orange-crowned Warbler","Northern Flicker");',con=cnx,parse_dates=['obsDt']) #Canada Jay filter is for testing purposes only
        ObsDataset['obsDt_week'] = ObsDataset['obsDt'].dt.isocalendar().week
        ObsDataset['howMany'] = ObsDataset['howMany'].fillna(1)
        #group up data
        ObsDataset = ObsDataset.groupby(['comName','StopName','locId','obsDt_week'])['howMany'].mean().reset_index()
        maxCount = ObsDataset.groupby(['comName'])['howMany'].max()
        #calculations
        ObsDataset['relativeAbundance'] = ObsDataset.apply(lambda x: (x.howMany/(maxCount[x.comName])),axis=1)  #baseline
        ObsDataset['relativeAbundance'] = ObsDataset.apply(lambda f: stopQualityMask(f.comName,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        maxAbd = ObsDataset.groupby(['comName'])['relativeAbundance'].max() #max abundance value by species
        ObsDataset['relativeAbundance'] = ObsDataset.apply(lambda n: (n.relativeAbundance/(maxAbd[n.comName])),axis=1)  #normalize
        ObsDataset.sort_values(by=['comName','obsDt_week'],ascending=True,inplace=True)
    except Exception as calcEx:
        raise calcEx
    finally: cnx.close()
    return ObsDataset[['comName','StopName','obsDt_week','locId','relativeAbundance']]

Forecasting Architecture:

- Reference models group birds based on expressed habitat preference, seasonality and rolling volume. Where neccessary, linear interpolation is applied to the reference models to reasonabily fill in gaps. The strength (re:limit value) of this interpolation and the accompanying mask is determined by rarity. Each bird+stop combination is assigned a reference model.
- R-squared testing assigns an interpolation method to each bird+stop combination to estimate the relative abundance of a bird at each stop based off of the given relative abundance and the reference model group
- Dependent forecasting uses existing relative abundance values to make inferences about relative abundance and habitat preference using a baseline of known values/sightings
- Independent forecasting projects the habitat preference of each bird over the habitat makeup of each stop, ignoring the sightings used to confirm the dependent forecasting model. For example: although there are not any Canada Jay sightings logged at Mt. Si, the habitat makeup of that stop correlates (Spearman rank) with the Canada Jay's preferred late-season habitat

In [ ]:
##TODO #77 identify birds that express seasonality/can only be found in the observed regions during specific times of year to inform the method of interpolation

In [369]:
##TODO #79 add 'StopName' as a loop layer to the relative abundance forecasting --DONE
def forecast_weeklyAbd():
    realObsAbd = weeklyAbundance()
    try:
        keyList = realObsAbd.drop_duplicates(subset=['comName','StopName'])
        realObsAbd = realObsAbd.groupby(['comName','StopName','obsDt_week'])['relativeAbundance'].mean().reset_index()
        forecastList = []
        for comName in keyList.itertuples():
            #species = pd.DataFrame(keyList[keyList['comName'] == comName.comName])
            obsAbd = realObsAbd[realObsAbd['comName'] == comName.comName]
            obsAbd = obsAbd[obsAbd['StopName'] == comName.StopName]
    #build presence time series scope
            weekMin = (obsAbd['obsDt_week'].min()-8)
            if weekMin <= 0: weekMin = 1
            weekMax = (obsAbd['obsDt_week'].max()+8)
            if weekMax > 53: weekMax = 53
            allweek = pd.DataFrame({'obsDt_week':range(weekMin,weekMax)})
            obsAbd.drop(columns=['comName','StopName'],inplace=True)
            ptForxt = pd.merge(left=obsAbd,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            ptForxt.set_index('obsDt_week',inplace=True)
            ptForxt.sort_index(axis='index',ascending=True,inplace=True)
    #mask in seasonality, where there are no birds present in the observed location
            ptForxt['mask'] = ptForxt['relativeAbundance'].interpolate(method='index',limit=5,limit_direction='both')  #mask, values do not matter
            ptForxt.loc[ptForxt['mask'].isna() == True,'relativeAbundance'] = 0
            ptForxt['forecastAbd'] = ptForxt['relativeAbundance'].interpolate(method='polynomial',order=5,limit=5,limit_direction='both')   #real forecast
            ptForxt['comName'] = comName.comName
            ptForxt['StopName'] = comName.StopName
            ptForxt = ptForxt.reset_index()
            forecastList.append(ptForxt)
        seasonalAbd = pd.DataFrame()
        seasonalAbd = pd.DataFrame(pd.concat(forecastList,ignore_index=True))
    except Exception as abdForxExc:
        raise abdForxExc
    return seasonalAbd

In [370]:
forx = forecast_weeklyAbd()

In [390]:
##TODO #78 calculate R2 scores for each relative abundance interpolation method

forecast = forx[['comName','StopName','obsDt_week','relativeAbundance']]
forecast = forecast[forecast['comName']=='Canada Jay']
forecast = forecast[forecast['StopName']=='ChiricoTrail_PooPooPoint']
forecast.set_index('obsDt_week',inplace=True)


In [381]:
from sklearn.metrics import r2_score

In [393]:
forecast = forecast.assign(fillMean=forecast.relativeAbundance.fillna(forecast.relativeAbundance.mean()))
forecast = forecast.assign(fillMedian=forecast.relativeAbundance.fillna(forecast.relativeAbundance.median()))
forecast = forecast.assign(rollingMean=forecast.relativeAbundance.fillna(forecast.relativeAbundance.rolling(24,min_periods=2,).mean()))
forecast = forecast.assign(linear=forecast.relativeAbundance.interpolate(method='linear',limit=5,limit_direction='both'))
forecast = forecast.assign(splineThree=forecast.relativeAbundance.interpolate(method='spline',order=3,limit=5,limit_direction='both'))
forecast = forecast.assign(polynomialThree=forecast.relativeAbundance.interpolate(method='polynomial',order=3,limit=5,limit_direction='both'))
forecast = forecast.assign(polynomialFive=forecast.relativeAbundance.interpolate(method='polynomial',order=5,limit=5,limit_direction='both'))
forecast

,comName,StopName,relativeAbundance,fillMean,fillMedian,rollingMean,linear,splineThree,splineFive,polynomialThree,polynomialFive
obsDt_week,,,,,,,,,,,
1,Canada Jay,ChiricoTrail_PooPooPoint,0.1250,0.1250,0.1250,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000
2,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.194444,0.323111,3.933743,1.745047,3.933743
3,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.263889,0.305552,5.724651,2.764517,5.724651
4,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.333333,0.288021,6.130639,3.274665,6.130639
5,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.402778,0.270576,5.659780,3.366745,5.659780
6,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.472222,0.253277,4.707637,3.132010,4.707637
7,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.541667,0.236182,3.569591,2.661714,3.569591
8,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.611111,0.219348,2.453168,2.047111,2.453168
9,Canada Jay,ChiricoTrail_PooPooPoint,NaN,0.1400,0.0000,NaN,0.680556,0.202836,1.490369,1.379455,1.490369


In [394]:
results = [(method,r2_score(forecast.relativeAbundance,forecast[method])) for method in list(forecast)[5:]]
results_df = pd.DataFrame(np.array(results),columns=['Method','r_squared'])
results_df.sort_values(by='r_squared',ascending=False)
results_df

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
##TODO #66 split out birds into quartiles based on the number of observations available and the YOY frequency. Results will determine the strength of forecasting required

In [71]:
##TODO #64 using the relative abundance value from weeklyAbundance(), write a function that returns FAO land coverage preference --DONE
##TODO #68 add seasons to relative abundance function, then add in a loop over each season in the function seasonalHabitatPreference
def seasonalHabitatPref():
    cnx = connectDB()
    try:
        wklyAbd = weeklyAbundance()
        #start season loop here
        rawList = []
        for locId in wklyAbd.itertuples():
            query = f'SELECT Barren,Urban,Grasslands,Savannas,waterBodies,evergreenNeedleleafForest,evergreenBroadleafForests,openForests,mixedBroadleafandNeedleleafForests,sparseForests,denseHerbaceous,sparseHerbaceous FROM FAO_by_locId WHERE FAO_by_locId.locId = "{locId.locId}"'
            lx = pd.read_sql(query,con=cnx)
            lx = lx.applymap(lambda x: locId.relativeAbundance*x,na_action='ignore')
            lx['comName'] = locId.comName
            lx['obsDt_week'] = locId.obsDt_week
            rawList.append(lx)
        results = pd.DataFrame()
        results = pd.concat(rawList,ignore_index=True)
        results = results.groupby(['comName','obsDt_week']).mean()
        results.reset_index(inplace=True)
        results = pd.DataFrame(results)
    except Exception as prefExc:
        raise UserWarning(prefExc)
    finally: cnx.close()
    return results

In [103]:
##TODO #65 using historic habitat preference data, forecast the seasonal habitat preference for each species of bird --DONE
##TODO #74 update the interpolation method used in forecasting habitat preference to polynomial or just use the forecasted relative abundance
def forecastHabtPref():
    realHabtPref = seasonalHabitatPref()
    habtKeys = realHabtPref.drop_duplicates(subset=['comName'])
    try:
        forxHabtList = []
        for i in habtKeys.itertuples():
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            obsPref = realHabtPref[realHabtPref['comName'] == i.comName]
            obsPref.drop(columns=['comName'],inplace=True)
            forxPref = pd.merge(left=obsPref,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            forxPref.set_index('obsDt_week',inplace=True)
            forxPref.sort_index(axis='index',ascending=True,inplace=True)
            #forxPref.interpolate(method='polynomial',order=3,inplace=True,limit=3,limit_direction='both')
            forxPref.interpolate(method='linear',inplace=True,limit=3,limit_direction='both')
            forxPref['comName'] = i.comName
            forxPref = forxPref.reset_index()
            forxHabtList.append(forxPref)
        forecastPref = pd.DataFrame()
        forecastPref = pd.concat(forxHabtList,ignore_index=True)
    except Exception as habtPrefForxExcp:
        raise UserWarning(habtPrefForxExcp)
    return forecastPref

In [ ]:
##TODO #65 using historic habitat preference data, forecast the seasonal habitat preference for each species of bird --DONE
##TODO #74 update the interpolation method used in forecasting habitat preference to polynomial or just use the forecasted relative abundance
def forecastHabtPref():
    realHabtPref = seasonalHabitatPref()
    habtKeys = realHabtPref.drop_duplicates(subset=['comName'])
    try:
        forxHabtList = []
        for i in habtKeys.itertuples():
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            obsPref = realHabtPref[realHabtPref['comName'] == i.comName]
            obsPref.drop(columns=['comName'],inplace=True)
            forxPref = pd.merge(left=obsPref,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            forxPref.set_index('obsDt_week',inplace=True)
            forxPref.sort_index(axis='index',ascending=True,inplace=True)
            #forxPref.interpolate(method='polynomial',order=3,inplace=True,limit=3,limit_direction='both')
            forxPref.interpolate(method='linear',inplace=True,limit=3,limit_direction='both')
            forxPref['comName'] = i.comName
            forxPref = forxPref.reset_index()
            forxHabtList.append(forxPref)
        forecastPref = pd.DataFrame()
        forecastPref = pd.concat(forxHabtList,ignore_index=True)
    except Exception as habtPrefForxExcp:
        raise UserWarning(habtPrefForxExcp)
    return forecastPref

In [104]:
forgex = forecastHabtPref()
forgex

C:\Users\lukew\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,obsDt_week,Urban,Savannas,evergreenNeedleleafForest,evergreenBroadleafForests,openForests,mixedBroadleafandNeedleleafForests,sparseForests,comName
0,1,NaN,NaN,0.010000,NaN,NaN,0.067692,NaN,Canada Jay
1,2,NaN,NaN,0.060171,NaN,NaN,0.067692,NaN,Canada Jay
2,3,NaN,NaN,0.110342,NaN,NaN,0.067692,NaN,Canada Jay
3,4,NaN,NaN,0.160513,NaN,NaN,0.067692,NaN,Canada Jay
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada Jay
...,...,...,...,...,...,...,...,...,...
100,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orange-crowned Warbler
101,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orange-crowned Warbler
102,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orange-crowned Warbler
103,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Orange-crowned Warbler
